In [ ]:
!pip install praw

In [ ]:
import praw
import pandas as pd

# Authenticate with Reddit API
reddit = praw.Reddit(
    client_id="h8bDtAzxHzVj-K4xpRuLWw",
    client_secret="HGDDO5rr43GWCxCyhWwW7B77K9SWuw",
    user_agent="Rscraper by IH_Datascientist"
)

# Scrape posts from a subreddit
subreddit = reddit.subreddit("stocks")
posts = []

for post in subreddit.new(limit=1000):
    posts.append({
        "title": post.title,
        "body": post.selftext,
        "score": post.score,
        "comments": post.num_comments,
        "created_utc": post.created_utc
    })

# Convert to DataFrame
df = pd.DataFrame(posts)
print(df.head())

In [ ]:
df.to_csv("reddit_stock_data.csv", index=False)

The provided Python script leverages the PRAW (Python Reddit API Wrapper) library to scrape data from the Reddit API.
2.	Defining the Target Subreddit:
The subreddit () method is used to specify the subreddit you want to scrape, in this case, "stocks."
3.	Scrapping Recent Posts:
To fetch the most recent posts from the subreddit, the subreddit.new() method used to fetch the most recent post on Reddit and also set limit=1000.
4.	Extracting Post Attributes:
To capture relevant data fields from each post, such as the title, content, score, number of comments, and creation time. Attributes like “post.title” and “post.selftext” are extracted for analysis. Data is stored in a dictionary, which is appended to a list (posts).
5.	Converting Data to Pandas DataFrame:
To structure the scraped data in a tabular format for easier manipulation and analysis.



In [ ]:
import pandas as pd
ds=pd.read_csv("/content/reddit_stock_data.csv")
ds.head()

,title,body,score,comments,created_utc
0,"VKTX holder, Thoughts on Adding CRBP to Portfolio",Hi all! \n\nI started investing in 2022 and on...,1,1,1.732120e+09
1,Understanding tax implications for internation...,So there's an Australian stock I'm planning on...,1,0,1.732119e+09
2,Target shares plunge 20% after discounter cuts...,[Target](https://www.cnbc.com/quotes/TGT/) on ...,151,121,1.732115e+09
3,The Secret to Compounding: Build Your War Ches...,"***“The market, like the Lord, helps those who...",0,3,1.732112e+09
4,Billionaire Warren Buffett Just Added Pool Cor...,"Warren Buffett, the legendary billionaire inve...",0,3,1.732112e+09


This step demonstrates loading raw data from a CSV file and Showing the top 5 rows of the dataset.


In [ ]:
import pandas as pd
from datetime import datetime

def clean_reddit_data(ds):
    # Check if columns exist before dropping NaNs
    subset_cols = ["title", "body"]
    existing_cols = [col for col in subset_cols if col in ds.columns]

    # Drop posts with missing titles and bodies, only if columns exist
    if existing_cols:
        ds.dropna(subset=existing_cols, inplace=True)

    # Combine 'title' and 'body' into a single column if columns exist
    if all(col in ds.columns for col in subset_cols):
        ds["content"] = ds["title"] + " " + ds["body"]
    else:
        # Handle the case where 'title' and 'body' columns are missing
        # You might want to create 'content' from other columns or skip it
        # For example, if you have a 'text' column, you can use:
        # ds["content"] = ds["text"]
        pass  # Or raise an error, or handle it differently

    # Convert timestamp to a readable date
    ds["created_date"] = pd.to_datetime(ds["created_utc"], unit="s").dt.date

    # Keep relevant columns
    # The 'id' column was not present in the original DataFrame
    # Replace 'id' with another existing column name if needed
    ds = ds[["content", "created_date","comments","score"]]
    return ds

ds = pd.read_csv("/content/reddit_stock_data.csv")  # Load the dataframe
ds = clean_reddit_data(ds)
print(ds)
ds.to_csv("cleaned_reddit_stock_data.csv", index=False)

                                               content created_date  comments  \
0    VKTX holder, Thoughts on Adding CRBP to Portfo...   2024-11-20         1   
1    Understanding tax implications for internation...   2024-11-20         0   
2    Target shares plunge 20% after discounter cuts...   2024-11-20       121   
3    The Secret to Compounding: Build Your War Ches...   2024-11-20         3   
4    Billionaire Warren Buffett Just Added Pool Cor...   2024-11-20         3   
..                                                 ...          ...       ...   
827  My first attempt at due diligence- Klaviyo (KV...   2024-09-12        17   
828  Adobe stock slips on soft fourth-quarter reven...   2024-09-12        17   
829  Large option activity at close - signal bearis...   2024-09-12         4   
830  What has the Russell 2000 been sluggish the pa...   2024-09-12        16   
831  Oracle jumps about 8% after hours after report...   2024-09-12        70   

     score  
0        1  
1

This step demonstrates the Removing rows with missing titles or body, Combining title and bodies into a single content field,
Converting Unix timestamps to a human-readable date format and
Retaining only relevant fields: content, created_date, comments, and score.

In [ ]:
ds.head()

,content,created_date,comments,score
0,"VKTX holder, Thoughts on Adding CRBP to Portfo...",2024-11-20,1,1
1,Understanding tax implications for internation...,2024-11-20,0,1
2,Target shares plunge 20% after discounter cuts...,2024-11-20,121,151
3,The Secret to Compounding: Build Your War Ches...,2024-11-20,3,0
4,Billionaire Warren Buffett Just Added Pool Cor...,2024-11-20,3,0


In [ ]:
import re

def preprocess_text(ds):
    def clean_text(text):
        # Remove URLs
        text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
        # Remove special characters and digits
        text = re.sub(r"[^a-zA-Z\s]", "", text)
        # Lowercase the text
        text = text.lower()
        # Remove extra spaces
        text = re.sub(r"\s+", " ", text).strip()
        return text

    # Apply cleaning to the content column
    ds["processed_content"] = ds["content"].apply(clean_text)
    return ds

# Example: Preprocess cleaned data
preprocessed_ds = preprocess_text(ds)
print(preprocessed_ds[["processed_content"]])

                                     processed_content
0    vktx holder thoughts on adding crbp to portfol...
1    understanding tax implications for internation...
2    target shares plunge after discounter cuts for...
3    the secret to compounding build your war chest...
4    billionaire warren buffett just added pool cor...
..                                                 ...
827  my first attempt at due diligence klaviyo kvyo...
828  adobe stock slips on soft fourthquarter revenu...
829  large option activity at close signal bearish ...
830  what has the russell been sluggish the past ye...
831  oracle jumps about after hours after reporting...

[831 rows x 1 columns]


This step demonstrates the process of text cleaning, which includes Removing URLs, special characters, and digits, Converting text to lowercase, Stripping extra spaces, and applying the cleaning function to the content field.

In [ ]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.2 MB/s eta 0:00:00


Installing the vaderSentiment for sentiment analysis

In [ ]:
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Perform sentiment analysis
def perform_sentiment_analysis(preprocessed_ds):
    analyzer = SentimentIntensityAnalyzer()
    preprocessed_ds["sentiment"] = preprocessed_ds["processed_content"].apply(lambda x: analyzer.polarity_scores(x)["compound"])
    return preprocessed_ds
preprocessed_ds.drop(columns=["content", "comments","score"], inplace=True)
# Add sentiment scores to the dataset
sentiment_data = perform_sentiment_analysis(preprocessed_ds)
print(sentiment_data)

# Save the dataset with sentiment scores
#sentiment_data.to_csv("reddit_with_sentiment.csv", index=False)

    created_date                                  processed_content  sentiment
0     2024-11-20  vktx holder thoughts on adding crbp to portfol...    -0.0027
1     2024-11-20  understanding tax implications for internation...     0.5499
2     2024-11-20  target shares plunge after discounter cuts for...     0.7579
3     2024-11-20  the secret to compounding build your war chest...    -0.9356
4     2024-11-20  billionaire warren buffett just added pool cor...     0.6124
..           ...                                                ...        ...
827   2024-09-12  my first attempt at due diligence klaviyo kvyo...     0.9661
828   2024-09-12  adobe stock slips on soft fourthquarter revenu...     0.9331
829   2024-09-12  large option activity at close signal bearish ...    -0.3818
830   2024-09-12  what has the russell been sluggish the past ye...     0.1280
831   2024-09-12  oracle jumps about after hours after reporting...     0.7706

[831 rows x 3 columns]


In this step, Sentiment scores are computed for the cleaned and preprocessed text using the VADER and stored in a new column Sentiment.

In [ ]:
sentiment_data.head()

,created_date,processed_content,sentiment
0,2024-11-20,vktx holder thoughts on adding crbp to portfol...,-0.0027
1,2024-11-20,understanding tax implications for internation...,0.5499
2,2024-11-20,target shares plunge after discounter cuts for...,0.7579
3,2024-11-20,the secret to compounding build your war chest...,-0.9356
4,2024-11-20,billionaire warren buffett just added pool cor...,0.6124


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = vectorizer.fit_transform(sentiment_data['processed_content'])
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(dtm)

print("Topics:")
for idx, topic in enumerate(lda.components_):
    print(f"Topic {idx+1}: {[vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]]}")

Topics:
Topic 1: ['years', 'price', 'market', 'time', 'company', 'stocks', 'just', 'im', 'like', 'stock']
Topic 2: ['stocks', 'including', 'earnings', 'eps', 'fed', 'fundamentals', 'inflation', 'rate', 'news', 'market']
Topic 3: ['share', 'growth', 'sales', 'company', 'earnings', 'year', 'quarter', 'million', 'revenue', 'billion']
Topic 4: ['chips', 'nvidia', 'intel', 'billion', 'companies', 'data', 'new', 'company', 'said', 'ai']
Topic 5: ['day', 'growth', 'shares', 'trading', 'options', 'price', 'news', 'click', 'stock', 'market']


This piece of code extracts hidden topics or themes from a collection of text documents and helps summarize large text datasets by identifying recurring patterns. It is often used in applications like sentiment analysis, document classification, content summarization, and recommendation systems.

In [ ]:
tickers = ['AAPL', 'TSLA', 'AMZN']
for ticker in tickers:
    sentiment_data[f'{ticker}_mentions'] = sentiment_data['processed_content'].str.count(ticker)

This code demonstrates how often each stock ticker appears in the processed text data (e.g., tweets, reviews, or articles).
It is useful for analyzing public sentiment or trends related to specific companies or stocks.

In [ ]:
import yfinance as yf

#Fetching historical data
stock_data = yf.download('AAPL', start='2024-01-01', end='2024-10-31')
stock_data['Price_Change'] = stock_data['Close'].pct_change()
stock_data['Label'] = (stock_data['Price_Change'] > 0).astype(int)
#stock_data.to_csv("stock_data.csv", index=False)

[*********************100%***********************]  1 of 1 completed


This step is used to fetch the historical stock price data for "AAPL",
computes the percentage change in the closing price (Price_Change) and
Assign a binary label:
1 if the price increased.
0 otherwise.

In [ ]:
stock_data = stock_data.reset_index()

In [ ]:
stock_data.head()

Price,Date,Adj Close,Close,High,Low,Open,Volume,Price_Change,Label
Ticker,,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,,
0,2024-01-02 00:00:00+00:00,184.734970,185.639999,188.440002,183.889999,187.149994,82488700,NaN,0
1,2024-01-03 00:00:00+00:00,183.351746,184.250000,185.880005,183.429993,184.220001,58414500,-0.007488,0
2,2024-01-04 00:00:00+00:00,181.023163,181.910004,183.089996,180.880005,182.149994,71983600,-0.012700,0
3,2024-01-05 00:00:00+00:00,180.296722,181.179993,182.759995,180.169998,181.990005,62303300,-0.004013,0
4,2024-01-08 00:00:00+00:00,184.655365,185.559998,185.600006,181.500000,182.089996,59144500,0.024175,1


In [ ]:
import pandas as pd

def merge_data(sentiment_data, stock_data):
    sentiment_data = pd.DataFrame(sentiment_data)
    # Convert 'created_date' to datetime64[ns] to match 'Date' column type
    sentiment_data['created_date'] = pd.to_datetime(sentiment_data['created_date'], infer_datetime_format=True)
    # Remove timezone information from 'created_date' to match 'Date'
    sentiment_data['created_date'] = sentiment_data['created_date'].dt.tz_localize(None)

    # Flatten the MultiIndex in stock_data if it has multiple levels
    stock_data.columns = stock_data.columns.get_level_values(0) if isinstance(stock_data.columns, pd.MultiIndex) else stock_data.columns

    # Convert 'Date' column to datetime if it's not already
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])

    # Remove timezone from stock_data['Date'] to align with sentiment_data['created_date']
    stock_data['Date'] = stock_data['Date'].dt.tz_localize(None)

    merged_df = pd.merge(sentiment_data, stock_data, left_on='created_date', right_on='Date', how='inner')
    return merged_df

This step demonstrates the process of Converting created_date in Reddit sentiment data and Date in stock market data to compatible datetime formats.
Merges the two datasets on their respective date columns.
Ensures only overlapping dates are included in the resulting dataset.

In [ ]:
dataset=merge_data(sentiment_data,stock_data)
print(dataset)


    created_date                                  processed_content  \
0     2024-10-30  doordash reports firstever quarterly profit re...   
1     2024-10-30  dropbox slashes of global workforce eliminatin...   
2     2024-10-30  starbucks ceo pledges to fundamentally change ...   
3     2024-10-30  meta beats on top and bottom lines but misses ...   
4     2024-10-30  microsoft reports fy q earnings beat across th...   
..           ...                                                ...   
460   2024-09-12  my first attempt at due diligence klaviyo kvyo...   
461   2024-09-12  adobe stock slips on soft fourthquarter revenu...   
462   2024-09-12  large option activity at close signal bearish ...   
463   2024-09-12  what has the russell been sluggish the past ye...   
464   2024-09-12  oracle jumps about after hours after reporting...   

     sentiment  AAPL_mentions  TSLA_mentions  AMZN_mentions       Date  \
0       0.9030              0              0              0 2024-10-30   

<ipython-input-13-0754ba8d0013>:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  sentiment_data['created_date'] = pd.to_datetime(sentiment_data['created_date'], infer_datetime_format=True)


In [ ]:
dataset.to_csv("Final_dataset.csv", index=False)

In [ ]:
dataset.head()

,created_date,processed_content,sentiment,AAPL_mentions,TSLA_mentions,AMZN_mentions,Date,Adj Close,Close,High,Low,Open,Volume,Price_Change,Label
0,2024-10-30,doordash reports firstever quarterly profit re...,0.9030,0,0,0,2024-10-30,229.847122,230.100006,233.470001,229.550003,232.610001,47070900,-0.015278,0
1,2024-10-30,dropbox slashes of global workforce eliminatin...,-0.1289,0,0,0,2024-10-30,229.847122,230.100006,233.470001,229.550003,232.610001,47070900,-0.015278,0
2,2024-10-30,starbucks ceo pledges to fundamentally change ...,0.9595,0,0,0,2024-10-30,229.847122,230.100006,233.470001,229.550003,232.610001,47070900,-0.015278,0
3,2024-10-30,meta beats on top and bottom lines but misses ...,0.7750,0,0,0,2024-10-30,229.847122,230.100006,233.470001,229.550003,232.610001,47070900,-0.015278,0
4,2024-10-30,microsoft reports fy q earnings beat across th...,0.9915,0,0,0,2024-10-30,229.847122,230.100006,233.470001,229.550003,232.610001,47070900,-0.015278,0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report

X = dataset[["sentiment","Volume"]]
y = dataset["Label"]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Apply SMOTE to the training set
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train_resampled, y_train_resampled)

# Evaluate on the test set
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.8924731182795699
              precision    recall  f1-score   support

           0       0.90      0.86      0.88        43
           1       0.88      0.92      0.90        50

    accuracy                           0.89        93
   macro avg       0.89      0.89      0.89        93
weighted avg       0.89      0.89      0.89        93



This code demonstrates a machine learning process for predicting stock price movements based on sentiment analysis. It splits the data into training and testing sets, using the sentiment and Volumes feature to predict the Label (whether the stock price increased or not). A RandomForestClassifier is trained on the training data (X_train, y_train), and predictions (y_pred) are made on the test set (X_test).SMOTE method used to address class imbalance in datasets. It generates synthetic samples for the minority class to Balance the dataset, improving model performance when the dataset is biased toward the majority class.
Model performance is evaluated using accuracy and a detailed classification report, which provides metrics like precision, recall, and F1-score for each class. This approach highlights the utility of sentiment analysis in financial predictions while providing insights into the model's strengths and weaknesses.